In [1]:
STOCK_NAME = '한화'

In [2]:
import numpy as np
import os
import pickle
import random
import datetime
from tqdm import tqdm
from multiprocessing import Pool
from MeCab import Tagger
import tensorflow as tf
import tensorflow.keras.backend as ktf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, Concatenate, Input, Embedding, Reshape
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint

## GPU Setting

In [3]:
# gpu 설정
def get_session():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5,allow_growth=True,visible_device_list='2')
    return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
ktf.set_session(get_session())

## Hyper Parameters

In [4]:
# 하이퍼파라미터
seq_size = 70  # model input shape[0], 입력 데이터의 형태소 최대 갯수
embed_size = 128  # model input shape[1], 각 형태소의 임베딩 벡터 크기
batch_size = 32 # 각 미니배치의 크리
vocab_size = 455001  # word2index_dict의 단어 수
validation_split = 0.1  # 학습 시 train set에서 validation set으로 사용할 데이터 비율
learning_rate = 0.001
dropout_rate = 0.5
kernel_sizes = [3, 4, 5]  # kernel_size list for each CNN layers
n_class = 1  # 분류할 클래스 수 (binary_crossentropy를 쓰므로 1개 클래스가 0 또는 1의 값을 가지는 것으로 2 클래스 분류)
n_epoch = 13
n_patience = 6  # early stop 콜백의 파라미터

# random seed
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

## 경로 설정

In [5]:
# 파일 경로
root_path = '/data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification'
data_path = f'{root_path}/data/sentences/increased_labeled/labeled_{STOCK_NAME}.txt'
vocab_path = f'{root_path}/data/vocabulary/word2index_dict_190117.pkl'
mecab_path = '-d /data/lib/mecab/mecab-ko-dic-default'

# 저장경로용 시간, 파일명 문자열
now_dt = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
model_name = data_path.split('/')[-1].replace('labeled_', '').replace('.txt', '')

# 텐서보드 디렉토리 생성 및 로그 경로
tensorboard_dir = 'tensorboard'
if not os.path.exists(tensorboard_dir):
    os.makedirs(tensorboard_dir)
tblog_path = f'{root_path}/{tensorboard_dir}/{model_name}'

# 체크포인트 디렉토리 생성
ckp_dir = f'{root_path}/checkpoint/{model_name}/'
if not os.path.exists(ckp_dir):
    os.makedirs(ckp_dir)
ckp_path = os.path.join(ckp_dir, now_dt + '_weights.{epoch:03d}-{val_acc:.4f}.hdf5')

## 단어사전 로드
word2index_dict.pkl을 로드한다. 거의 대부분의 단어들을 유니크한 숫자로 인코딩하기 위한 딕셔너리이다. (str -> int)

In [6]:
with open(vocab_path, 'rb') as fp:
    word2index_dict = pickle.load(fp)
    
if vocab_size != len(word2index_dict):
    vocab_size = len(word2index_dict)
print(len(word2index_dict))

455001


## 데이터 로드


In [7]:
def load_data(path):
    with open(data_path, 'r') as fp:
        data = [l.strip() for l in fp.readlines() if len(l) > 10 and len(l.split())]
    data_X = [d.rsplit('#', 1)[0] for d in data]
    data_y = [int(d.rsplit('#', 1)[-1]) for d in data]
    return data_X, data_y

In [8]:
data_X, data_y = load_data(data_path)
print('data_X size : ', len(data_X))
print('data_y size : ', len(data_y))
print('data_X[10] : ', data_X[10])
print('data_y[10] : ', data_y[10])

data_X size :  49267
data_y size :  49267
data_X[10] :  다이와넥스트은행의 설립 당시 자 본금은 500억엔(한화 4492억원)이었다
data_y[10] :  0


### !!! 클래스별 데이터 수 비교하기



In [9]:
# 레이블별 데이터 비율 비교
class_0_data = [x for x, y in zip(data_X, data_y) if y == 0]
class_1_data = [x for x, y in zip(data_X, data_y) if y == 1]

print(f'class_0_data size : {len(class_0_data)}')
print(f'class_1_data size : {len(class_1_data)}')

class_0_data size : 9561
class_1_data size : 39706


In [ ]:
# sample_x_c0 = class_0_data
# sample_x_c1 = random.sample(class_1_data, 40000)
# data_X = sample_x_c0 + sample_x_c1

## Tokenizing

In [10]:
def tokenize(sentence):
    tagger = Tagger(mecab_path)
    raw_tokens = tagger.parse(sentence).splitlines()
    parsed_tokens = []
    for raw_token in raw_tokens:
        word_tag_ruple = raw_token.split('\t')
        if len(word_tag_ruple) != 2:
            continue
        word_stem = word_tag_ruple[0]
        word_tag = word_tag_ruple[1].split(',')[0]
        if not word_stem or not word_tag:
            continue
        if word_tag in {'NNP', 'NNG', 'SL', 'VV', 'VA', 'VX'}:
            parsed_tokens.append(word_stem)
    return parsed_tokens

In [11]:
tokenized_sents = []
with Pool(processes=8) as pool:
    for tokens in tqdm(pool.imap(tokenize, data_X), total=len(data_X), desc='tokenizing'):
        if tokens:
            tokenized_sents.append(tokens)
            
print('tokenized_sents size : ', len(tokenized_sents))

tokenizing: 100%|██████████| 49267/49267 [00:08<00:00, 5659.59it/s]

tokenized_sents size :  49267


## Encoding

In [12]:
def encode(tokens):
    padded_tokens = list(map(lambda x : tokens[x] if x < len(tokens) else '#', range(seq_size)))
    embed_vect = list(map(lambda w : word2index_dict[w] if w in word2index_dict else 0, padded_tokens))
    return embed_vect

In [13]:
encode_X = []
with Pool(processes=8) as pool:
    for encoded_tokens in tqdm(pool.imap(encode, tokenized_sents), total=len(data_X), desc='encoding'):
        encode_X.append(encoded_tokens)
        
print('encode_X size : ', len(encode_X))

encoding: 100%|██████████| 49267/49267 [00:03<00:00, 15087.91it/s]

encode_X size :  49267


## Split Train - Test Dataset

In [14]:
def get_train_test_data(data_x, data_y):
    
    x_c0, x_c1 = [], []
    for x, y in zip(data_x, data_y):
        if y == 0:
            x_c0.append(x)
        if y == 1:
            x_c1.append(x)
    
    increased_c1 = []
    
    if len(x_c0) >= len(x_c1)*2:
        div_ratio = int(len(x_c0) / len(x_c1))
        for i in range(div_ratio):
            if  (i + 1) * len(x_c1) < len(x_c0):
                increased_c1 += x_c1
    else:
        increased_c1 = x_c1
            
    shuff_c0 = random.sample(x_c0, len(x_c0))
    shuff_c1 = random.sample(increased_c1, len(increased_c1))
    
    c0_tsize = int(len(shuff_c0) * validation_split)
    c1_tsize = int(len(shuff_c1) * validation_split)
    
    test_X = shuff_c0[:c0_tsize] + shuff_c1[:c1_tsize]
    train_X = shuff_c0[c0_tsize:] + shuff_c1[c1_tsize:]
    test_y = [0 for _ in range(c0_tsize)] + [1 for _ in range(c1_tsize)]
    train_y = [0 for _ in range(len(shuff_c0) - c0_tsize)] + [1 for _ in range(len(shuff_c1) - c1_tsize)]    
        
    return np.array(train_X), np.array(train_y), np.array(test_X), np.array(test_y)


In [15]:
train_X, train_y, test_X, test_y = get_train_test_data(encode_X, data_y)

print(f'train_X size : {len(train_X)}')
print(f'train_y size : {len(train_y)}')
print(f'test_X size : {len(test_X)}')
print(f'test_y size : {len(test_y)}')

train_X size : 44341
train_y size : 44341
test_X size : 4926
test_y size : 4926


## Model

In [16]:
def get_char_cnn_model(kernel_sizes, seq_size, vocab_size, embed_size, dropout_rate, n_class, learning_rate):
    # input layer
    inputs = Input(shape=(seq_size,), dtype='float32', name='input')
    
    embedded = Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=seq_size, name='embedded')(inputs)
    reshaped = Reshape((seq_size, embed_size, 1), name='reshape')(embedded)

    # multiple CNN layers
    convolution_layers = []
    for idx, kernel_size in enumerate(kernel_sizes):
        conv = Conv2D(filters=256, kernel_size=(kernel_size, embed_size), padding='valid', activation='relu', name=f'conv_{idx}')(reshaped)
        pool = MaxPooling2D(pool_size=(seq_size - kernel_size + 1, 1), strides=(1, 1), padding='valid', name=f'pool_{idx}')(conv)
        dropout = Dropout(rate=dropout_rate, name=f'dropout_{idx}')(pool)
        convolution_layers.append(dropout)
        
    # concatenate all CNN output tensors
    concat_tensor = Concatenate(axis=1, name='concat')(convolution_layers)    
    
    # middle fully-connected layer
    flatten = Flatten(name='flatten')(concat_tensor)
    hidden = Dense(384, activation='relu', name='hidden')(flatten)
    dropout = Dropout(rate=dropout_rate, name='dropout')(hidden)
    
    # output layer
    outputs = Dense(n_class, activation='sigmoid', name='output')(dropout)

    # connect layers to model
    model = Model(inputs=inputs, outputs=outputs)
    
    # compile model
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [17]:
model = get_char_cnn_model(kernel_sizes, seq_size, vocab_size, embed_size, dropout_rate, n_class, learning_rate)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 70)           0                                            
__________________________________________________________________________________________________
embedded (Embedding)            (None, 70, 128)      58240128    input[0][0]                      
__________________________________________________________________________________________________
reshape (Reshape)               (None, 70, 128, 1)   0           embedded[0][0]                   
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 68, 1, 256)   98560       reshape[0][0]                    
__________________________________________________________________________________________________
conv_1 (Co

## Callbacks

In [18]:
early_stopping = EarlyStopping(patience=n_patience)
# tensorboard = TensorBoard(log_dir=tblog_path, batch_size=batch_size)
checkpoint = ModelCheckpoint(ckp_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_weights_only=False)

In [19]:
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir=tblog_path, **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()
        
tensorboard = TrainValTensorBoard(batch_size=batch_size)

## Training

In [20]:
history = model.fit(
    train_X, 
    train_y, 
    batch_size=batch_size, 
    epochs=n_epoch,
    validation_data=(test_X, test_y),
    callbacks=[early_stopping, tensorboard, checkpoint]
)

/data/venv/lib64/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 44341 samples, validate on 4926 samples
Epoch 1/13
44341/44341 [==============================] - 37s 840us/step - loss: 0.1328 - acc: 0.9491 - val_loss: 0.0339 - val_acc: 0.9898

Epoch 00001: val_loss improved from inf to 0.03390, saving model to /data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification/checkpoint/한화/20190308172117_weights.001-0.9898.hdf5
Epoch 2/13
44341/44341 [==============================] - 35s 793us/step - loss: 0.0332 - acc: 0.9892 - val_loss: 0.0256 - val_acc: 0.9905

Epoch 00002: val_loss improved from 0.03390 to 0.02562, saving model to /data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification/checkpoint/한화/20190308172117_weights.002-0.9905.hdf5
Epoch 3/13
44341/44341 [==============================] - 35s 791us/step - loss: 0.0205 - acc: 0.9932 - val_loss: 0.0268 - val_acc: 0.9931

Epoch 00003: val_loss did not improve from 0.02562
Epoch 4/13
44341/44341 [==============================] - 35s 789us/step - loss: 0.0174 - acc: 0.9947 - va

## Predict

In [21]:
loaded_model = tf.keras.models.load_model('/data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification/checkpoint/한화/20190308172117_weights.002-0.9905.hdf5')

/data/venv/lib64/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [22]:
def predict(model, sentence):
    tokens = tokenize(sentence)
    encodes = encode(tokens)
    tf_input = np.array(encodes).reshape((1, seq_size))
    pred = model.predict(tf_input)[0][0]
    result = 1 if pred >= 0.85 else 0
    return result, pred

In [23]:
text = '한화에너지(대표이사 류두형)는 미국 하와이 전력청(HECO)이 주관한 태양광+에너지저장장치(ESS) 발전소 건설 및 운영사업 입찰'
pred = predict(loaded_model, text)
pred

(1, 0.99987507)

In [25]:
text = '[보안뉴스 박미영 기자] 한화는 지난 17일 종합연구소(대전 유성구 장동 소재)에서 직접 보고 들은 선진 방산 기술을 공유하는 글로벌 방산 기술 공유 자리를 마련했다'
pred = predict(loaded_model, text)
pred

(1, 0.9954542)

In [26]:
text = '한화테크윈은 “일상 곳곳의 안전보안 솔루션(Expansion of Secure Life)”을 주제로 전시부스를 체험형 부스로 기획했다'
pred = predict(loaded_model, text)
pred

(1, 0.99936086)

In [24]:
text = '탄탄해진 안방은 한화 이글스가 2018시즌 가을야구에 진출한 비결 중 하나였다. '
pred = predict(loaded_model, text)
pred

(0, 0.002326496)

In [27]:
text = '2018년 11년 만의 가을야구 진출에 성공한 한화이글스가 팀 역사상 유일무이한 첫 번째 우승을 한 게 1999년이다.'
pred = predict(loaded_model, text)
pred

(0, 0.00033649872)

In [28]:
text = '열심히 일한 직원들 위해 연말 보너스로 현금 500억 원 쏜 통 큰 사장님 ... 바로 3억 위안(한화 약 500억 원)에 달하는 현금을 연말 보너스로 직원들'
pred = predict(loaded_model, text)
pred

(0, 7.1868286e-05)

In [29]:
text = '선수들은 월급 1,250달러(한화 약 140만 원)와 상금 5만 5,125달러(한화 약 6,216만 원)가 지급되지 않았다'
pred = predict(loaded_model, text)
pred

(0, 0.00010316949)

In [30]:
text = '하지만 여름 이적 시장을 통해 사파타가 1400만 유로(한화 약 180억)의 금액과 함께 아탈란타로 임대를 떠나면서 삼프도리아가 자랑하던 영혼'
pred = predict(loaded_model, text)
pred

(0, 0.00036213495)

In [31]:
text = '한화 대전공장 폭발사고 희생자 유가족들이 김승연 한화그룹 회장과의 면담을 요구했지만 성사되지 않았다'
pred = predict(loaded_model, text)
pred

(1, 0.9999974)

In [32]:
text = '유가족들은 8일 오전 서울 장교동 한화빌딩 앞에서 기자회견을 열고 회사의 무책임한 태도를 규탄했다'
pred = predict(loaded_model, text)
pred

(1, 0.9818885)

In [33]:
text = '기자회견 직후 유가족들은 김승연 회장과의 면담을 위해 한화빌딩 1층 로비로 진입했다'
pred = predict(loaded_model, text)
pred

(1, 0.9275667)

In [34]:
text = '이에 최선목 한화커뮤니케이션위원회 위원장은 유가족들에게 "대화할 수 있는 회의실을 준비했다"며 "김승연 회장은 이 사안에 관여할 권한이 없다"고 설명했다'
pred = predict(loaded_model, text)
pred

(0, 0.27977183)

In [35]:
text = '협상에는 유가족 대표 3명과 옥경석 한화 화약방산부문 대표이사, 최선목 위원장 총 5명이 나선다'
pred = predict(loaded_model, text)
pred

(1, 0.99999917)

In [36]:
text = '유가족들이 김 회장과의 면담을 요구하며 강하게 반발하고 나선 이유는 한화 측의 작업 매뉴얼 조작 의혹 때문이다'
pred = predict(loaded_model, text)
pred

(0, 0.021755055)

In [37]:
text = "한화제약(사장 김경락)이 급·만성호흡기질환 치료제 '뮤테란'의 시럽 제형을 이달 2일 출시했다고 밝혔다"
pred = predict(loaded_model, text)
pred

(0, 0.43083677)

In [38]:
text = '뮤테란은 점액의 분비 운동력을 높여 화농성과 농성 점액 분비물의 점도를 최대한 낮춰 계면 활성층을 보강해 원활한 분비조절 작용을 촉진시키며, 점액의 수송 속도와 섬모운동의 횟수를 증가시키고, 기도의 청정화로 기침을 완화, 점액과 섬모계의 생리적 방어기능을 회복시켜 점막기능이 정상화 될 수 있도록 도와준다고 한화제약은 설명했다'
pred = predict(loaded_model, text)
pred

(0, 0.018281424)

In [39]:
text = '메니피’는 1998년부터 1년간 미국에서 경주마로 활동할 당시 11번의 경주를 거쳐 약 173만 미국달러(한화 약 19억 원)를 벌었지만, 씨수말로 전환 후 자마들의 상금 총합이 574억 원으로 놀라운 부가 가치를 창출하고 있다'
pred = predict(loaded_model, text)
pred

(0, 0.0010890788)

In [40]:
text = '나홀로집에’ 케인이 사용한 금액 967달러는 한화라면 100만원도 넘는 큰 금액이다'
pred = predict(loaded_model, text)
pred

(0, 3.656097e-06)

In [41]:
text = '그렇다면 2000년 ~ 2018년 시즌까지 체결된 FA 계약 201회 중에 원 소속팀에 남은 선수는 어느 정도나 될까'
pred = predict(loaded_model, text)
pred

(0, 0.070523284)